In [5]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
from tqdm import tqdm

In [81]:
#load road network for helsinki for driving
G = ox.graph_from_place("Helsinki, Finland", network_type='drive')

#simplify graph by removing duplicates
#graph_to_gdfs: Convert a MultiGraph or MultiDiGraph to node and/or edge GeoDataFrames.
#docs chnaged, no need to use graph_to_gdfs() if you use graph_from_place
#G = ox.graph_to_gdfs(G)

#print(type(G))          # Should be networkx.MultiDiGraph
print(G)                # Or check if it's a tuple

MultiDiGraph with 9801 nodes and 21209 edges


In [85]:
#load hexes with population and centroid
hex_grid = gpd.read_parquet("../data/processed/hex_with_population.parquet")
hex_grid = hex_grid.to_crs(epsg=3857)  #match OSMnx graph CRS
print(hex_grid.crs)

EPSG:3857


In [87]:
#compute hex centroid in lat/lon if not ready
hex_grid['center'] = hex_grid.geometry.centroid

In [ ]:
hospitals = ox.features_from_place("Helsinki, Finland", tags={"amenity": "hospital"})
hospitals = hospitals.to_crs(epsg=3857)
hospitals = hospitals[hospitals.geometry.type == "Point"]

In [ ]:
hospital_nodes = hospitals.geometry.apply(lambda p: ox.distance.nearest_nodes(G, p.x, p.y)).tolist()

In [ ]:

#function to find shortest travel time
def shortest_travel_time(point):
    try:
        orig_node = ox.distance.nearest_nodes(G, point.x, point.y)
        times = []
        for hospital_node in hospital_nodes:
            length = nx.shortest_path_length(G, orig_node, hospital_node, weight='length')  # meters
            speed_kmh = 30  #assuming avg urban speed
            time_min = (length / 1000) / speed_kmh * 60
            times.append(time_min)
        return min(times) if times else None
    except:
        return None

In [115]:
#calculate travel time
tqdm.pandas()
hex_grid['travel_time_min'] = hex_grid['center'].progress_apply(shortest_travel_time)

100%|██████████████████████████████████████████████████████████████████████████████| 4790/4790 [04:05<00:00, 19.50it/s]


In [99]:
hex_grid.to_parquet("../data/processed/hex_with_travel_time.parquet")

In [83]:
type(G)

networkx.classes.multidigraph.MultiDiGraph

In [105]:
print(len(hospital_nodes))


3


In [107]:
hex_grid['travel_time_min'].describe()


count    282.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: travel_time_min, dtype: float64

In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry import Point
from tqdm import tqdm

In [ ]:
G = ox.graph_from_place("Helsinki, Finland", network_type='drive')
G = ox.project_graph(G, to_crs="EPSG:3857") 



In [ ]:
hex_grid = gpd.read_parquet("../data/processed/hex_with_population.parquet")
hex_grid = hex_grid.to_crs(epsg=3857) 

In [ ]:
hex_grid['center'] = hex_grid.geometry.centroid  

In [ ]:
hospitals = ox.features_from_place("Helsinki, Finland", tags={"amenity": "hospital"})
hospitals = hospitals[~hospitals.geometry.is_empty]
hospitals = hospitals.to_crs(epsg=3857) 
hospitals['geometry'] = hospitals.geometry.centroid  

In [ ]:
hospital_nodes = hospitals.geometry.apply(lambda p: ox.distance.nearest_nodes(G, p.x, p.y)).tolist()

In [ ]:
def shortest_travel_time(point):
    try:
        orig_node = ox.distance.nearest_nodes(G, point.x, point.y)
        times = []
        for hospital_node in hospital_nodes:
            length = nx.shortest_path_length(G, orig_node, hospital_node, weight='length')  # in meters
            speed_kmh = 30  # Assume average city driving speed
            time_min = (length / 1000) / speed_kmh * 60
            times.append(time_min)
        return min(times) if times else None
    except Exception as e:
        return None

In [ ]:
tqdm.pandas()
hex_grid['travel_time_min'] = hex_grid['center'].progress_apply(shortest_travel_time)

100%|██████████████████████████████████████████████████████████████████████████████| 4790/4790 [13:53<00:00,  5.75it/s]


In [ ]:
hex_grid.to_parquet("../data/processed/hex_with_travel_time.parquet")

In [136]:
hex_grid['travel_time_min'].describe()

count    4770.000000
mean       13.893624
std         7.073382
min         0.000000
25%         6.924899
50%        17.510040
75%        19.371198
max        34.717531
Name: travel_time_min, dtype: float64